In [16]:
import pandas as pd

df_raw = pd.read_excel("../data/KPMG Tax Case - Data Set.xlsx")
df_raw.drop_duplicates(inplace=True)
df_raw.reset_index(drop=True,inplace=True)
df_raw

,Date,Title,Numac,Link FR,Link NL
0,2020-01-14,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
1,2020-01-16,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
2,2020-01-16,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
3,2020-01-24,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
4,2020-01-28,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
...,...,...,...,...,...
782,2022-04-04,SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021....,2.022021e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
783,2022-04-04,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONC...",2.022202e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
784,2022-04-04,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MA...,2.022031e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
785,2022-04-04,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGI...,4.000213e+07,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...


In [19]:

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import html2text
import requests

df = df_raw.copy(deep=True)

df['cleantextnl'] = np.nan


pnt=0
language = 'nl'
for url in df['Link NL']:
    
    #Link is correct
    if type(url) != float:
        #generate the correct url
        htmllanguage = 'language=' + language
        url = url.replace('language=nl', htmllanguage)
        url2 = url.replace('.pl?', '_body.pl?')
        
        #try to access the url 
        try:
            response = requests.get(url2,timeout=5)

            #Get the text from the html
            soup = BeautifulSoup(response.content, "html").text
            dummy = html2text.html2text(soup)

            dummy = dummy.split('---|---|---|---|---|---')
            dummy = dummy[0].split('begin |  |  eerste woord |  laatste woord |  |')
            
            cleantext = dummy[0]

            #Input the cleaned text into the dataframe & Save it to file
            txtlabel = 'cleantext' + language
            df[txtlabel][pnt] = cleantext
            pnt +=1
            
            df.to_pickle("../data/Staatsblad.pkl")
        
        #if the connections times out continue with the rest of the urls
        except requests.RequestException:
            print("scraping :", pnt,url)
            print("RequestException")
            pnt +=1
            continue
        
    #Link is Nan        
    else:
        print("scraping :", pnt,url)
        print("NAN")
        pnt +=1

#Save final dataframe
df.reset_index(drop=True,inplace=True)
print(df)
df.to_pickle("../data/Staatsblad.pkl")      
print("Done Scraping")

C:\Users\Frede\AppData\Local\Temp\ipykernel_19292\3682959143.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[txtlabel][pnt] = cleantext


scraping : 755 nan
NAN
scraping : 773 nan
NAN
          Date                                              Title  \
0   2020-01-14  REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...   
1   2020-01-16  SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...   
2   2020-01-16  SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...   
3   2020-01-24  MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...   
4   2020-01-28  SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...   
..         ...                                                ...   
782 2022-04-04  SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021....   
783 2022-04-04  SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONC...   
784 2022-04-04  SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MA...   
785 2022-04-04  GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGI...   
786 2022-04-05  GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nCOMM...   

            Numac                                            Link FR  \
0    2.020010e+09  http://www.ejustice.just.fgov.be/c

In [20]:
df = pd.read_pickle("../data/Staatsblad.pkl")
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df

,Date,Title,Numac,Link FR,Link NL,cleantextnl
0,2020-01-14,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR bel...
1,2020-01-16,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR DE ...
2,2020-01-16,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR DE ...
3,2020-01-24,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR ein...
4,2020-01-28,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR bel...
...,...,...,...,...,...,...
765,2022-04-04,SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021....,2.022021e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR DE ...
766,2022-04-04,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONC...",2.022202e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR bel...
767,2022-04-04,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MA...,2.022031e+09,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL FR bel...
768,2022-04-04,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGI...,4.000213e+07,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,Moniteur Belge - Belgisch Staatsblad NL\n\n
